Czyszczenie danych
czyszcenie zostało przeprowadzone tlyko dla prouktów i sesji, gdyż, w pliku z użytkownikami nie zostały znalezione błędy.

W przypadku sesji, uzupełniliśmy brakujące dane o użytkownikach, na podstawie pozostałych wierszy gdzie była ta sama sesja oraz identyfikator użytkownika.
Natomiast te wiersze, gdzie product_id było nullem, zostały usunięte.

Jeśli chodzi o produkty, usunęliśmy te ze skrajnymi lub ujemnymi cenami.

In [1]:
from json_handler.json_handler import load_jsonl_data
from preprocessing.junk_remover import JunkRemover
import pandas as pd


sessions = load_jsonl_data("data/input/sessions.jsonl")
products = load_jsonl_data("data/input/products.jsonl")
jr = JunkRemover()

clean_sessions, clean_products = jr.delete_junk_data(sessions, products)
df_sessions = pd.DataFrame(clean_sessions)


df_sessions.head()
df_sessions.isnull().sum()



session_id              0
timestamp               0
user_id                 0
product_id              0
event_type              0
offered_discount        0
purchase_id         49805
dtype: int64

In [2]:
df_products = pd.DataFrame(clean_products)
df_products.head()

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99


nastepnie kodujemy kolumne event_type na wartosci binarne 0,1 i usuwamy jeszcze zbędną kolumnę purchase_id

In [3]:
from sklearn.preprocessing import LabelEncoder

df_sessions = pd.DataFrame(clean_sessions)
df_sessions.head(5)


,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2020-04-28T19:48:07,102,1040,VIEW_PRODUCT,0,NaN
1,100001,2020-04-28T19:51:46,102,1041,VIEW_PRODUCT,0,NaN
2,100001,2020-04-28T19:55:49,102,1042,VIEW_PRODUCT,0,NaN
3,100001,2020-04-28T19:58:35,102,1043,VIEW_PRODUCT,0,NaN
4,100001,2020-04-28T20:01:40,102,1045,VIEW_PRODUCT,0,NaN


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_events = le.fit_transform(df_sessions['event_type'])

df_sessions = df_sessions.drop(columns = ['event_type'])
df_sessions["event_type"] = encoded_events

df_sessions.head(5)

,session_id,timestamp,user_id,product_id,offered_discount,purchase_id,event_type
0,100001,2020-04-28T19:48:07,102,1040,0,NaN,1
1,100001,2020-04-28T19:51:46,102,1041,0,NaN,1
2,100001,2020-04-28T19:55:49,102,1042,0,NaN,1
3,100001,2020-04-28T19:58:35,102,1043,0,NaN,1
4,100001,2020-04-28T20:01:40,102,1045,0,NaN,1


zapisujemy przygotowane dane do pliku

In [5]:
import json
import os

directory = './prepared_data'
if not os.path.exists(directory):
    os.makedirs(directory)



out = df_sessions.to_json(orient='records')[1:-1].replace('},{', '}\n{')


with open('./prepared_data/prepared_sessions.jsonl', 'w') as session_file:
    session_file.write(out)
with open('./prepared_data/preapred_products.json', 'w') as product_file:
    json.dump(clean_products,product_file, indent=4)